In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

In [3]:
pip install sklearn

  Using cached sklearn-0.0.post1.tar.gz (3.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for sklearn: started
  Running setup.py install for sklearn: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: sklearn is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


In [2]:
# Классический набор данных для проверки качества работы алгоритмов классификации
from sklearn.datasets import load_iris

ModuleNotFoundError: No module named 'sklearn'

In [8]:
sns.set(rc={'figure.figsize': (11.7, 8.27)})

In [9]:
def custom_compare(x, y):
    if str(x) != str(y):
        raise RuntimeError(f'Ожидаемое значение: {y}. Фактическое: {x}')

# Логистическая регрессия

In [10]:
def skal(v1, v2):
    v1, v2 = np.array(v1), np.array(v2)
    return sum(v1 * v2)

def ziped(l, n):
    return [get_col(l, i) for i in range(n)]

def get_col(matrix, i):
    return [col[i] for col in matrix]

def p(w,x):
    return 1/(1+np.exp(-skal(x,w)))
def round_to_2(x):
    return round(x, 2)

In [13]:
def logistic_regression_predict_solution(w, data, factor_names):
    n = len(data[factor_names[0]])
    xs = np.array(ziped([[1] * n, *[np.array(data[name]) for name in factor_names]], n))
    ps = np.array([round_to_2(p(w,x)) for x in xs])
    return ps

In [5]:
def logistic_regression_predict_test():
    w_example_1 = np.array([0.1, 1])
    data_example_1 = pd.DataFrame({
        'x': [0.22, -0.41],
    })
    
    res_example_1 = np.array([0.58, 0.42])
    
    custom_compare(logistic_regression_predict_solution(w_example_1, data_example_1, ['x']), res_example_1)
    
    w_example_2 = np.array([0.1, 2.7, 2.3, -4.1])
    data_example_2 = pd.DataFrame({
        'x': [0.58, 0.15],
        'extra': [1, 2],
        'y': [0.58, 0.19],
        'z': [0.93, 0.44]
    })
    
    res_example_2 = np.array([0.31, 0.3])
    
    custom_compare(logistic_regression_predict_solution(w_example_2, data_example_2, ['x', 'y', 'z']), res_example_2)

    print('Тест прошёл успешно!')

In [14]:
logistic_regression_predict_test()

Тест прошёл успешно!


In [11]:
def logistic_summary_loss_gradient_solution(w, X, y):
    n = len(y)
    grad = 0 
    for i in range(n):
        grad += y[i] * sum([-y[i]*skal(X[i],w) for i in range(n)])
    grad*=-1
    print(grad)

In [16]:
def logistic_summary_loss_gradient_test():
    w_example_1 = np.array([-0.2, -0.3])
    X_example_1 = np.array([
        [1, -2.1],
        [1, 3.7]
    ])

    y_example_1 = np.array([-1, -1])
    
    res_example_1 = np.array([0.82, -0.49])
    
    custom_compare(logistic_summary_loss_gradient_solution(w_example_1, X_example_1, y_example_1), res_example_1)
    
    w_example_2 = np.array([0.3, -0.1, 0.4])
    X_example_2 = np.array([
        [1, 0.1, 0.3],
        [1, -0.2, 0.5]
    ])

    y_example_2 = np.array([1, -1])
    
    res_example_2 = np.array([0.23, -0.17,  0.19])
    
    custom_compare(logistic_summary_loss_gradient_solution(w_example_2, X_example_2, y_example_2), res_example_2)

    print('Тест прошёл успешно!')

In [ ]:
logistic_summary_loss_gradient_test()

In [13]:
def logistic_regression_solve_solution(data, factor_names, y_name, 
                                       learning_rate=0.01, eps=0.1):
    """
    С помощью градиентного спуска строит модель логистической регрессии по переданному набору данных.
    
    Аргументы:
        data: Таблица с объектами обучающей выборки.
              Каждый объект описывается набором численных факторов. 
              В данных может быть представлено больше факторов, чем модель должна использовать для предсказания. 
              Искусственного константного фактора, который для всех объектов равен 1 и 
              который будет использоваться моделью для предсказания, в таблице нет.
        factor_names: Список названий факторов, которые модель должна использовать для предсказания.
        y_name: Название столбца таблицы, в котором для каждого объекта содержится значение предсказываемого класса.
                Класс может иметь значение либо -1, либо 1.
        learning_rate: Опциональный параметр. Коэффициент скорости обучения, который используется в алгоритме градиентного спуска.
        eps: Опциональный параметр. Минимальное расстояние между текущей точкой градиентного спуска и следующей,
             при котором работа алгоритма останавливается.
        
    Возвращаемое значение:
        Возвращает вектор весов модели. 
        Координата вектора с индексом 0 соответствует свободному коэффициенту модели.
        Координата вектора с индексом i соответствует фактору с индексом i - 1 в списке factor_names.
    """
    
    X = data[factor_names].to_numpy()
    y = data[y_name].to_numpy()
    
    ones = np.ones((len(y), 1))
    n = len(y)
    
    X = np.hstack([ones, X])

    # Необходимо задать стартовый набор весов, который является начальной
    # точкой для алгоритма градиентного спуска.
    # В качестве стартового набора весов необходимо использовать вектор, состоящий из значений 0
    w_cur = np.zeros(n)
    
    while True:
        # Вычисление градиента с помощью функции logistic_summary_loss_gradient_solution.
        # Важно убрать округление результата работы функции до 2 знаков после запятой
        gradient_value = logistic_summary_loss_gradient_solution(w_cur, X, y)
    
        # Полезно уменьшить значение градиента, разделив каждую его координату на число объектов
        # в выборке, на которой происходит обучение модели
        gradient_value /= len(y)
        
        # Классический шаг градиентного спуска: переход из текущей точки в направлении,
        # противоположном вектору градиента
        w_new = w_cur - gradient_value * learning_rate
        
        # Проверка того, что расстояние между текущей точкой и новой не стало меньше или равно eps
        if np.linalg.norm(w_new - w_curr) <= eps:
            break
            
        w_cur = w_new
    
    return w_cur.round(2)

In [5]:
def logistic_regression_solve_test():
    data_example_1 = pd.DataFrame({
        'x1': [0.3, -0.1, -0.1, 0.4],
        'x2': [0.1, -0.1, 0.2, -0.1],
        'y': [1, -1, -1, 1],
    })

    factor_names_example_1 = ['x1', 'x2']
    y_name_example_1 = 'y'
    
    res_example_1 = np.array([-0.07, 0.94, -0.1])
    
    custom_compare(logistic_regression_solve_solution(data_example_1, factor_names_example_1, y_name_example_1,
                                                      learning_rate=0.001, eps=0.0001),
                  res_example_1)
    
    
    iris = load_iris()
    
    y_example_2 = (iris.target >= 1).astype('int64').reshape((len(iris.target), 1))
    y_example_2[y_example_2 == 0] = -1
    
    factor_names_example_2 = ['x1', 'x2']
    y_name_example_2 = 'y'
    
    data_example_2 = pd.DataFrame(
        columns=['x1', 'x2', 'x3', 'x4'] + [y_name_example_2],
        data=np.hstack([iris.data, y_example_2])
    )
    
    res_example_2 = np.array([-0.23,  1.17, -1.83])
    
    custom_compare(logistic_regression_solve_solution(data_example_2, factor_names_example_2, y_name_example_2,
                                                      learning_rate=0.01, eps=0.001),
                   res_example_2)
    
    print('Тест прошёл успешно!')

In [15]:
def logistic_regression_solve_res():
    # Загрузка набора данных для тестирования алгоритмов классификации
    iris = load_iris()
    
    # Приведение классов, которые необходимо научиться предсказывать, к значениям -1 и 1
    y = (iris.target > 1).astype('int64').reshape((len(iris.target), 1))
    y[y == 0] = -1
    
    # Создание таблицы на основе набора данных.
    # Факторы, которые есть в данных, будут называться 'x1', 'x2', 'x3' и 'x4'.
    # Классы объектов помещаются в колонку 'y'
    data = pd.DataFrame(
        columns=['x1', 'x2', 'x3', 'x4', 'y'],
        data=np.hstack([iris.data, y])
    )
    
    # Для предсказания будут использоваться только факторы 'x1' и 'x2'
    factor_names = ['x1', 'x2']
    # Предсказываемая характеристика — 'y'
    y_name = 'y'
    
    # Определение оптимальных весов для разработанной модели логистической регрессии
    ws = logistic_regression_solve_solution(data, factor_names, y_name,
                                            learning_rate=0.01, eps=0.001)
    
    for i in range(len(ws)):
        print(f'w{i}:', ws[i])

In [18]:
logistic_regression_solve_test()

ValueError: operands could not be broadcast together with shapes (3,) (4,) 

In [17]:
logistic_regression_solve_res()

w0: -0.23
w1: 1.17
w2: -1.83
